In [1]:
# 组合：直接把需要的类，放进去实例化，就可以了。

In [4]:
# 一个水池，水池里有乌龟和鱼。
# 乌龟和鱼，不是同一个物种，不用继承，用组合。

class Turtle:
    def __init__(self,x):
        self.num = x
        
class Fish:
    def __init__(self, x):
        self.num = x
    
class Pool:
    def __init__(self, x, y):
        self.turtle = Turtle(x)
        self.fish = Fish(y)
    def print_num(self):
        print("水池里共有乌龟%d只，小鱼%d条。"%(self.turtle.num,self.fish.num))


pool = Pool(10,100)
pool.print_num()

水池里共有乌龟10只，小鱼100条。


In [5]:
# Mixin机制

# Mixin简介
    # Mixin编程是一种开发模式，
    # 是一种将多个类中的功能单元，进行组合利用的方式，
    # 这听起来是类的继承机制就可以实现，但与传统的类继承不同，
    # Mixin并不作为任何类的基类，也不关心与什么类一起使用，
    # 而是在运行时，动态的同其他类一起组合使用。

# Mixin特点
    # 可以在不修改任何源代码的情况下，对已有类进行扩展；
    # 可以保证组件的划分；
    # 可以根据需要，使用已有的功能进行组合，来实现新的“类”；
    # 避免类继承的局限性，因为新的业务需求，可能需要创建新的子类。
    


In [7]:
# Mixin机制与多继承
# 可以利用Python的多继承，方便的实现Mixin机制。

# A，B 分别表示不同的功能单元，C为A，B功能的组合，则C就拥有A，B的功能。

class A:
    def get_a(self):
        print("a")
        
class B:
    def get_b(self):
        print("b")
    
class C(A,B):
    pass

c = C()
c.get_a()
c.get_b()


a
b


In [15]:
# __bases__
# 多继承的实现，会创建新类。
# 有时，在运行时，希望给类A添加类B的功能时，可以使用python的元编程特性，
# __bases__ 属性可以在运行时，轻松的给类A添加类B的特性。


class A:
    def get_a(self):
        print("a")
        
class B:
    def get_b(self):
        print("b")
    
A.__bases__ += (B,)
a.get_b()

TypeError: Cannot create a consistent method resolution
order (MRO) for bases object, B

In [16]:
# __bases__
# 其实 __bases__ 也是继承的机制，因为 __bases__ 属性存储了类的基类。
# 因此，多继承方法，也可以这样实现。

class C:
    pass

C.__bases__ += (A,B, )

TypeError: Cannot create a consistent method resolution
order (MRO) for bases object, A, B

In [ ]:
# 以上两个 __bases__ 示例在Python3.x 中错误，其原因是：
# Python3.x 中的类默认是新式类，在新式类中，无法修改 __bases__

# 在Python2.x中，分新式类和旧式类，旧式类可以正常运行。


In [14]:
# 以上两种方式，都是基于多继承和python的元编程特性，
# 然而在业务需求变化时，就需要新的功能组合，
# 那么就需要重新修改A的基类，这会带来同步的问题，
# 因为我们改的是类的特性，而不是对象的。
# 因此以上修改会对所有引用该类的模块都收到影响，这是相当危险的。
# 通常我们希望修改对象的行为，而不是修改类的。
# 同样的我们可以利用__dict__来扩展对象的方法。


class PlugIn(object):
    def __init__(self):
        self._exported_methods = []
        
    def plugin(self, owner):
        for f in self._exported_methods:
            owner.__dict__[f.__name__] = f

    def plugout(self, owner):
        for f in self._exported_methods:
            del owner.__dict__[f.__name__]

class AFeature(PlugIn):
    def __init__(self):
        super(AFeature, self).__init__()
        self._exported_methods.append(self.get_a_value)

    def get_a_value(self):
        print('a feature.')

class BFeature(PlugIn):
    def __init__(self):
        super(BFeature, self).__init__()
        self._exported_methods.append(self.get_b_value)

    def get_b_value(self):
        print('b feature.')

class Combine:pass

c = Combine()
AFeature().plugin(c)
BFeature().plugin(c)

c.get_a_value()
c.get_b_value()

a feature.
b feature.
